# Day 7 : No Space Left On Device

## Part 1


In [1]:
from backend import *

testInput = parseInput("""$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k""", parseMethod=parseWords)

__________ Input to be parsed __________
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
... and maybe more
____________________
__________ Parsed input __________
('$', 'cd', '/')
('$', 'ls')
('dir', 'a')
('14848514', 'b.txt')
('8504156', 'c.dat')
... and maybe more
____________________


The actual data:

In [2]:
actualInput = parseInput(open("inputs/day7.txt").read(), parseMethod=parseWords)

__________ Input to be parsed __________
$ cd /
$ ls
dir plws
dir pwlbgbz
dir pwtpltr
... and maybe more
____________________
__________ Parsed input __________
('$', 'cd', '/')
('$', 'ls')
('dir', 'plws')
('dir', 'pwlbgbz')
('dir', 'pwtpltr')
... and maybe more
____________________


In [ ]:
class Folder():
    def __init__(self, path : str, items : list, parentItem = None):
        self.path = path
        self.items = items
        self.parentItem = parentItem

    def folderSize(self) -> int:
        return(sum(map(sizeOfPathElement, self.items)))

    @classmethod
    def isInFolder(cls, searchFolder, searchedFolder):
        return searchFolder in searchedFolder
    
    def findFolder(self, pathName):
        found = [item for item in self.items if item.path == pathName]
        return found[0] if len(found) >= 1 else None
    
    def listAllFolders(self):
        subFolders = [self]
        if all(map(lambda W : type(W) == File, self.items)): return([self])
        for item in self.items:
            if type(item) == type(self) : subFolders.extend(item.listAllFolders())
        return subFolders
            
class File():
    def __init__(self, path : str, fileSize : int):
        self.path = path
        self.fileSize = fileSize

def sizeOfPathElement(pathElement : File | Folder) -> int:
    """Returns the size of a path element which is either a file or folder"""
    if type(pathElement) == File : return pathElement.fileSize
    if type(pathElement) == Folder : return pathElement.folderSize()

def instructionsToTree(instructions : list) -> Folder:
    """Iterate through an instructionset and return the main folder"""
    rootFolder = Folder("root", [])
    currentFolder = rootFolder
    lsList = [] ## To keep track of LS
    for firstElement, *additionalElements in instructions:
        if firstElement == "$" : # We perform a new action
            command, *commandArguments = additionalElements
            if command == "cd" :
                ## Clear the lsList, and add the items to the current folder
                currentFolder.items.extend(lsList)
                lsList = []
                if commandArguments[0] == "/" : currentFolder = rootFolder; continue
                if commandArguments[0] == ".." : currentFolder = currentFolder.parentItem; continue
                newFolder = currentFolder.findFolder(f"{currentFolder.path}/{commandArguments[0]}")
                if not newFolder : ## Add the folder to the structure if not found
                    newFolder = Folder(f"{currentFolder.path}/{commandArguments[0]}", [], currentFolder)
                    currentFolder.items.append(newFolder)
                currentFolder = newFolder
        else : # We are still listing items
            fileSize, fileName = firstElement, additionalElements
            if fileSize == "dir" : continue # Do not add directories to the folder, this is indexed when changing directories.
            lsList.append(File(fileName, int(fileSize)))
    
    currentFolder.items.extend(lsList) # Take care of final items if they exist
    return rootFolder


95437

In [89]:
day7_part1 = lambda W : sum([folder.folderSize() for folder in Folder.listAllFolders(instructionsToTree(W)) if folder.folderSize() <= 1e5])
test(day7_part1, testInput, 95437)

Answer: 95437            0.000075 seconds
Test succeeded.


In [90]:
run(day7_part1, actualInput)

Answer: 1432936            0.002431 seconds


1432936

## Part 2

In [ ]:
def day7_part2(input):
    folderSizes = list(map(Folder.folderSize, Folder.listAllFolders(instructionsToTree(input))))
    requiredSpace = 3e7 - 7e7 + max(folderSizes)
    return(min([folderSize for folderSize in folderSizes if folderSize >= requiredSpace]))

24933642

In [116]:
test(day7_part2, testInput, 24933642)

Answer: 24933642            0.000063 seconds
Test succeeded.


In [117]:
run(day7_part2, actualInput)

Answer: 272298            0.001884 seconds


272298